In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import json
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')


Libraries imported.


#### The dataset "NYPD Motor Vehicle Collisions" can be accssed either via downloading the CSV (which is a big file) or by using the JSON endpoint. The latter was chosen. 

In [34]:
df_ny = pd.read_json('https://data.cityofnewyork.us/resource/qiz3-axqb.json')

#### Do exploratory analysis and find out how the data looks like.

In [35]:
df_ny.head()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,4.0,16860.0,39.0,72.0,3.0,QUEENS,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,25-63 STEINWAY STREET,2019-02-09,40.767532,"{'type': 'Point', 'coordinates': [-73.912025, ...",-73.912025,0,0,0,0,1,0,1,0,NaN,NaN,2:10,4078201,Sedan,NaN,NaN,NaN,NaN,11103.0
1,32.0,11723.0,70.0,4.0,4.0,MANHATTAN,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.712646,"{'type': 'Point', 'coordinates': [-73.98998, 4...",-73.989980,0,0,0,0,0,0,0,0,RUTGERS STREET,MADISON STREET,13:20,4078103,Pick-up Truck,Sedan,NaN,NaN,NaN,10002.0
2,35.0,10932.0,49.0,23.0,5.0,BRONX,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.807190,"{'type': 'Point', 'coordinates': [-73.92431, 4...",-73.924310,0,0,0,0,0,0,0,0,EAST 135 STREET,WILLIS AVENUE,17:43,4078265,Taxi,Sedan,NaN,NaN,NaN,10454.0
3,25.0,17215.0,45.0,47.0,2.0,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.655760,"{'type': 'Point', 'coordinates': [-73.87319, 4...",-73.873190,0,0,0,0,0,0,0,0,VANDALIA AVENUE,ELTON STREET,16:05,4078543,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11208.0
4,10.0,12077.0,12.0,3.0,4.0,NaN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.742380,"{'type': 'Point', 'coordinates': [-74.00882, 4...",-74.008820,0,0,0,0,0,0,0,0,NaN,11 AVENUE,18:23,4077650,Box Truck,Sedan,NaN,NaN,NaN,NaN


In [36]:
df_ny.describe()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,latitude,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,unique_key,zip_code
count,926.000000,920.000000,926.000000,926.000000,926.000000,928.000000,928.000000,1000.00000,1000.0,1000.000000,1000.0,1000.000000,1000.0,1000.000000,1000.0,1.000000e+03,621.000000
mean,26.942765,14946.110870,37.273218,43.225702,3.150108,40.726817,-73.915246,0.01200,0.0,0.204000,0.0,0.053000,0.0,0.269000,0.0,4.077768e+06,10872.610306
std,14.694289,3864.033363,20.842054,22.246283,1.128565,0.076754,0.085668,0.10894,0.0,0.702052,0.0,0.232906,0.0,0.728814,0.0,5.012381e+02,540.780424
min,1.000000,10090.000000,1.000000,1.000000,1.000000,40.526894,-74.209060,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,4.076794e+06,10000.000000
25%,14.000000,12073.000000,20.000000,26.000000,2.000000,40.670178,-73.972318,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,4.077383e+06,10454.000000
50%,27.500000,14190.000000,39.000000,44.500000,3.000000,40.726011,-73.923755,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,4.077666e+06,11208.000000
75%,39.000000,17214.000000,56.750000,64.000000,4.000000,40.770777,-73.861392,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,4.078179e+06,11354.000000
max,51.000000,24670.000000,71.000000,77.000000,5.000000,40.907402,-73.710140,1.00000,0.0,12.000000,0.0,2.000000,0.0,12.000000,0.0,4.079096e+06,11693.000000


In [37]:
df_ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 34 columns):
:@computed_region_92fq_4b7q      926 non-null float64
:@computed_region_efsh_h5xi      920 non-null float64
:@computed_region_f5dn_yrer      926 non-null float64
:@computed_region_sbqj_enih      926 non-null float64
:@computed_region_yeji_bk3q      926 non-null float64
borough                          622 non-null object
contributing_factor_vehicle_1    996 non-null object
contributing_factor_vehicle_2    831 non-null object
contributing_factor_vehicle_3    61 non-null object
contributing_factor_vehicle_4    11 non-null object
contributing_factor_vehicle_5    4 non-null object
cross_street_name                243 non-null object
date                             1000 non-null datetime64[ns]
latitude                         928 non-null float64
location                         928 non-null object
longitude                        928 non-null float64
number_of_cyclist_injured        1

In [38]:
df_ny.shape

(1000, 34)

#### Now that we have a little idea about the dataset, it appears there are quite a few NaN values, as seen in cell[10]. For analysis, these should be excluded.

In [39]:
df_ny.dropna()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code


In [40]:
df_ny.head()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,4.0,16860.0,39.0,72.0,3.0,QUEENS,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,25-63 STEINWAY STREET,2019-02-09,40.767532,"{'type': 'Point', 'coordinates': [-73.912025, ...",-73.912025,0,0,0,0,1,0,1,0,NaN,NaN,2:10,4078201,Sedan,NaN,NaN,NaN,NaN,11103.0
1,32.0,11723.0,70.0,4.0,4.0,MANHATTAN,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.712646,"{'type': 'Point', 'coordinates': [-73.98998, 4...",-73.989980,0,0,0,0,0,0,0,0,RUTGERS STREET,MADISON STREET,13:20,4078103,Pick-up Truck,Sedan,NaN,NaN,NaN,10002.0
2,35.0,10932.0,49.0,23.0,5.0,BRONX,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.807190,"{'type': 'Point', 'coordinates': [-73.92431, 4...",-73.924310,0,0,0,0,0,0,0,0,EAST 135 STREET,WILLIS AVENUE,17:43,4078265,Taxi,Sedan,NaN,NaN,NaN,10454.0
3,25.0,17215.0,45.0,47.0,2.0,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.655760,"{'type': 'Point', 'coordinates': [-73.87319, 4...",-73.873190,0,0,0,0,0,0,0,0,VANDALIA AVENUE,ELTON STREET,16:05,4078543,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11208.0
4,10.0,12077.0,12.0,3.0,4.0,NaN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.742380,"{'type': 'Point', 'coordinates': [-74.00882, 4...",-74.008820,0,0,0,0,0,0,0,0,NaN,11 AVENUE,18:23,4077650,Box Truck,Sedan,NaN,NaN,NaN,NaN


#### Whoops! It appears every row has atleast one NaN value, which has erased all the data we had. This means we have to be more specific in removing the NaN values. This can be done by just removing them from the columns we are interested in for the time being. To find out which borough in New York has the most accidents, we will just focus on that for now.

In [41]:
df_ny.dropna(subset=['borough'])

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,4.0,16860.0,39.0,72.0,3.0,QUEENS,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,25-63 STEINWAY STREET,2019-02-09,40.767532,"{'type': 'Point', 'coordinates': [-73.912025, ...",-73.912025,0,0,0,0,1,0,1,0,NaN,NaN,2:10,4078201,Sedan,NaN,NaN,NaN,NaN,11103.0
1,32.0,11723.0,70.0,4.0,4.0,MANHATTAN,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.712646,"{'type': 'Point', 'coordinates': [-73.98998, 4...",-73.989980,0,0,0,0,0,0,0,0,RUTGERS STREET,MADISON STREET,13:20,4078103,Pick-up Truck,Sedan,NaN,NaN,NaN,10002.0
2,35.0,10932.0,49.0,23.0,5.0,BRONX,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.807190,"{'type': 'Point', 'coordinates': [-73.92431, 4...",-73.924310,0,0,0,0,0,0,0,0,EAST 135 STREET,WILLIS AVENUE,17:43,4078265,Taxi,Sedan,NaN,NaN,NaN,10454.0
3,25.0,17215.0,45.0,47.0,2.0,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.655760,"{'type': 'Point', 'coordinates': [-73.87319, 4...",-73.873190,0,0,0,0,0,0,0,0,VANDALIA AVENUE,ELTON STREET,16:05,4078543,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11208.0
5,21.0,14510.0,66.0,68.0,3.0,QUEENS,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,NaN,2019-02-09,40.741734,"{'type': 'Point', 'coordinates': [-73.85811, 4...",-73.858110,0,0,0,0,0,0,0,0,52 AVENUE,103 STREET,12:50,4078067,Sedan,NaN,NaN,NaN,NaN,11368.0
6,27.0,13514.0,14.0,48.0,2.0,BROOKLYN,Passing Too Closely,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.676693,"{'type': 'Point', 'coordinates': [-73.99683, 4...",-73.996830,0,0,0,0,0,0,0,0,SMITH STREET,LUQUER STREET,10:58,4077977,Sedan,Sedan,NaN,NaN,NaN,11231.0
7,33.0,16858.0,53.0,66.0,3.0,QUEENS,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.742874,"{'type': 'Point', 'coordinates': [-73.933945, ...",-73.933945,0,0,0,0,0,0,0,0,47 AVENUE,VANDAM STREET,17:30,4077577,Station Wagon/Sport Utility Vehicle,Pick-up Truck,NaN,NaN,NaN,11101.0
9,25.0,17215.0,45.0,47.0,2.0,BROOKLYN,Unspecified,NaN,NaN,NaN,NaN,NaN,2019-02-09,40.666290,"{'type': 'Point', 'coordinates': [-73.87268, 4...",-73.872680,0,0,0,0,0,0,0,0,LOGAN STREET,LINDEN BOULEVARD,13:30,4078524,Sedan,NaN,NaN,NaN,NaN,11208.0
10,16.0,24336.0,63.0,63.0,3.0,QUEENS,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2019-02-09,40.729940,"{'type': 'Point', 'coordinates': [-73.74481, 4...",-73.744810,0,0,0,0,0,0,0,0,SPRINGFIELD BOULEVARD,HILLSIDE AVENUE,20:00,4077626,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11427.0
11,47.0,24018.0,63.0,63.0,3.0,QUEENS,Unspecified,NaN,NaN,NaN,NaN,NaN,2019-02-09,40.680832,"{'type': 'Point', 'coordinates': [-73.732925, ...",-73.732925,0,0,1,0,0,0,1,0,234 STREET,129 AVENUE,16:00,4077579,Sedan,NaN,NaN,NaN,NaN,11422.0


#### Much better, isn't it? The number of rows has come down from 1000 to 594, which means we have 594 entries in the column borough. It is sufficient to make a judgement on one which has the least amount of accidents.

In [42]:
df_ny['borough'].value_counts()

BROOKLYN         197
QUEENS           182
MANHATTAN        121
BRONX            101
STATEN ISLAND     21
Name: borough, dtype: int64

#### Now that is quite a difference. It appears that the safest neighbourhood is Staten Island!

In [43]:
df1= df_ny[['borough', 'contributing_factor_vehicle_1']]

In [44]:
df1.dropna(subset=['borough'])

,borough,contributing_factor_vehicle_1
0,QUEENS,Driver Inattention/Distraction
1,MANHATTAN,Following Too Closely
2,BRONX,Passing or Lane Usage Improper
3,BROOKLYN,Unspecified
5,QUEENS,Driver Inattention/Distraction
6,BROOKLYN,Passing Too Closely
7,QUEENS,Unspecified
9,BROOKLYN,Unspecified
10,QUEENS,Unspecified
11,QUEENS,Unspecified


In [45]:
df1['contributing_factor_vehicle_1'].value_counts()

Driver Inattention/Distraction                           261
Unspecified                                              224
Following Too Closely                                     94
Failure to Yield Right-of-Way                             71
Backing Unsafely                                          52
Passing or Lane Usage Improper                            44
Passing Too Closely                                       38
Other Vehicular                                           32
Unsafe Lane Changing                                      30
Reaction to Uninvolved Vehicle                            23
Turning Improperly                                        21
Unsafe Speed                                              18
Pavement Slippery                                         14
Driver Inexperience                                       14
Traffic Control Disregarded                               12
Alcohol Involvement                                       11
View Obstructed/Limited 

In [46]:
df2=df1.groupby(['borough','contributing_factor_vehicle_1']).size()

In [47]:
print(df2)


borough        contributing_factor_vehicle_1                        
BRONX          Alcohol Involvement                                       2
               Backing Unsafely                                          5
               Driver Inattention/Distraction                           19
               Driver Inexperience                                       3
               Failure to Yield Right-of-Way                             7
               Fatigued/Drowsy                                           1
               Following Too Closely                                     6
               Glare                                                     1
               Other Vehicular                                           9
               Passing Too Closely                                       5
               Passing or Lane Usage Improper                            3
               Pedestrian/Bicyclist/Other Pedestrian Error/Confusion     1
               Reaction to Unin

#### Staten Island had 21 collisions, out of which backing unsafely was the most frequent, as can be seen above.
#### Now we should explore the neighbourhood of Staten Island.

In [48]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [49]:
newyork_data


{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [50]:
neighborhoods_data = newyork_data['features']

In [51]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### This data has to be transformed into a Pandas dataframe

In [52]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [53]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [54]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [55]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [56]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [57]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


#### Let's look at the neighborhoods in Staten Island.

In [58]:
Staten_Island_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
Staten_Island_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


#### Finding out the geographical coordinates for Staten Island

In [59]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Staten Island are 40.5834557, -74.1496048.


#### Let's visualize Staten Island

In [60]:
map_staten_island = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Staten_Island_data['Latitude'], Staten_Island_data['Longitude'], Staten_Island_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_staten_island)  
    
map_staten_island

#### Now we can use the Foursquare API to segment the neighborhoods

In [61]:
CLIENT_ID = 'CPJQDUVBZNDEWN3QV0BQ0N2GSWKKPOBNIEHPR0QE5PNULOXQ' # your Foursquare ID
CLIENT_SECRET = 'UJC41OEBQQCKYHYYBOURJXTHCFHJU1G5Q0RQ0REFJXZ2JHU1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CPJQDUVBZNDEWN3QV0BQ0N2GSWKKPOBNIEHPR0QE5PNULOXQ
CLIENT_SECRET:UJC41OEBQQCKYHYYBOURJXTHCFHJU1G5Q0RQ0REFJXZ2JHU1


#### Now we can explore the first neighborhood in the dataframe

In [62]:
Staten_Island_data.loc[0, 'Neighborhood']

'St. George'

#### St. George is the first neighborhood

In [63]:
neighborhood_latitude = Staten_Island_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Staten_Island_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Staten_Island_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. George are 40.6449815710044, -74.07935312512797.


#### Now we can get the list of top 10 venues in St. George within a radius of 1000 meters

In [64]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=CPJQDUVBZNDEWN3QV0BQ0N2GSWKKPOBNIEHPR0QE5PNULOXQ&client_secret=UJC41OEBQQCKYHYYBOURJXTHCFHJU1G5Q0RQ0REFJXZ2JHU1&v=20180605&ll=40.6449815710044,-74.07935312512797&radius=1000&limit=100'

In [65]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c694600dd57977bd44ae358'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bf9c5c08d30d13a6bce0218-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_monument_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d12d941735',
         'name': 'Monument / Landmark',
         'pluralName': 'Monuments / Landmarks',
         'primary': True,
         'shortName': 'Landmark'}],
       'id': '4bf9c5c08d30d13a6bce0218',
       'location': {'cc': 'US',
        'city': 'Staten Island',
        'country': 'United States',
        'distance': 311,
        'formattedAddress': ['Staten Island, NY 10301', 'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.64676748643786,
          'lng':

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the JSON and structure it into a Pandas dataframe

In [67]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510
1,Beso,Tapas Restaurant,40.643306,-74.076508
2,Richmond County Bank Ballpark,Baseball Stadium,40.645056,-74.076864
3,A&S Pizzeria,Pizza Place,40.643940,-74.077626
4,Enoteca Maria,Italian Restaurant,40.641941,-74.077320


In [68]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

70 venues were returned by Foursquare.


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### We need to make a dataframe titled 'Staten Island Venues' which will give us all the nearby venues by using the above function

In [70]:
staten_island_venues = getNearbyVenues(names=Staten_Island_data['Neighborhood'],
                                   latitudes=Staten_Island_data['Latitude'],
                                   longitudes=Staten_Island_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


#### Check the size of the dataframe

In [71]:
print(staten_island_venues.shape)
staten_island_venues.head()

(807, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
1,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
2,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
3,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
4,St. George,40.644982,-74.079353,St. George Theatre,40.642253,-74.077496,Theater


#### Let's check how many venues were returned for each neighborhood

In [72]:
staten_island_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,13,13,13,13,13,13
Arden Heights,5,5,5,5,5,5
Arlington,7,7,7,7,7,7
Arrochar,18,18,18,18,18,18
Bay Terrace,12,12,12,12,12,12
Bloomfield,4,4,4,4,4,4
Bulls Head,43,43,43,43,43,43
Butler Manor,6,6,6,6,6,6
Castleton Corners,16,16,16,16,16,16


#### We can narrow it down to just unique values

In [73]:
print('There are {} uniques categories.'.format(len(staten_island_venues['Venue Category'].unique())))

There are 169 uniques categories.


#### Now we can analyze each neighborhood

In [74]:
# one hot encoding
staten_island_onehot = pd.get_dummies(staten_island_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
staten_island_onehot['Neighborhood'] = staten_island_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [staten_island_onehot.columns[-1]] + list(staten_island_onehot.columns[:-1])
staten_island_onehot = staten_island_onehot[fixed_columns]

staten_island_onehot.head()

,Women's Store,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lawyer,Liquor Store,Market,Martial Arts Dojo,Mattress Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Optical Shop,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [75]:
staten_island_onehot.shape

(807, 169)

In [76]:
staten_island_grouped = staten_island_onehot.groupby('Neighborhood').mean().reset_index()
staten_island_grouped

,Neighborhood,Women's Store,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lawyer,Liquor Store,Market,Martial Arts Dojo,Mattress Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint
0,Annadale,0.000000,0.000000,0.076923,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.076923,0.00000,0.0,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000
1,Arden Heights,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000

In [77]:
staten_island_grouped.shape

(63, 169)

#### Print each neighborhood with the top ten most common venues

In [78]:
num_top_venues = 10

for hood in staten_island_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = staten_island_grouped[staten_island_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annadale----
              venue  freq
0       Pizza Place  0.15
1      Dance Studio  0.08
2            Bakery  0.08
3     Deli / Bodega  0.08
4    Cosmetics Shop  0.08
5              Park  0.08
6        Sports Bar  0.08
7             Diner  0.08
8  Sushi Restaurant  0.08
9        Restaurant  0.08


----Arden Heights----
                     venue  freq
0              Pizza Place   0.2
1                 Pharmacy   0.2
2              Coffee Shop   0.2
3                     Pool   0.2
4               Playground   0.2
5               Nail Salon   0.0
6  New American Restaurant   0.0
7             Optical Shop   0.0
8                     Park   0.0
9    Performing Arts Venue   0.0


----Arlington----
                 venue  freq
0             Bus Stop  0.29
1         Intersection  0.14
2        Grocery Store  0.14
3                 Food  0.14
4        Boat or Ferry  0.14
5  American Restaurant  0.14
6           Kids Store  0.00
7               Lawyer  0.00
8         Optical Shop  0.00


9            Pharmacy  0.08


----Howland Hook----
                     venue  freq
0            Moving Target   1.0
1                 Pharmacy   0.0
2              Music Venue   0.0
3               Nail Salon   0.0
4  New American Restaurant   0.0
5             Optical Shop   0.0
6                     Park   0.0
7    Performing Arts Venue   0.0
8      Peruvian Restaurant   0.0
9                Pet Store   0.0


----Huguenot----
                     venue  freq
0                     Bank   0.1
1       Italian Restaurant   0.1
2  Comfort Food Restaurant   0.1
3               Donut Shop   0.1
4           Sandwich Place   0.1
5         Asian Restaurant   0.1
6              Bridal Shop   0.1
7            Train Station   0.1
8           Ice Cream Shop   0.1
9                   Lawyer   0.1


----Lighthouse Hill----
                     venue  freq
0       Italian Restaurant  0.25
1               Art Museum  0.25
2                      Spa  0.25
3                     Café  0.25
4            

                     venue  freq
0                     Pier  0.33
1            Deli / Bodega  0.33
2                    Beach  0.17
3       Athletics & Sports  0.17
4            Women's Store  0.00
5  New American Restaurant  0.00
6             Optical Shop  0.00
7                     Park  0.00
8    Performing Arts Venue  0.00
9      Peruvian Restaurant  0.00


----St. George----
                 venue  freq
0        Deli / Bodega  0.09
1                  Bar  0.09
2     Baseball Stadium  0.04
3      Harbor / Marina  0.04
4  Monument / Landmark  0.04
5                 Park  0.04
6          Pizza Place  0.04
7           Playground  0.04
8                Plaza  0.04
9           Donut Shop  0.04


----Stapleton----
                   venue  freq
0            Pizza Place  0.09
1         Cosmetics Shop  0.09
2             Restaurant  0.06
3         Discount Store  0.06
4         Sandwich Place  0.06
5                   Bank  0.06
6                   Café  0.03
7         Breakfast Spot  0.0

In [79]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [80]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = staten_island_grouped['Neighborhood']

for ind in np.arange(staten_island_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(staten_island_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Pizza Place,Dance Studio,Cosmetics Shop,Bakery,Park,Diner,Restaurant,Sports Bar,Train Station,American Restaurant
1,Arden Heights,Pool,Coffee Shop,Playground,Pizza Place,Pharmacy,Wings Joint,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Arlington,Bus Stop,Grocery Store,American Restaurant,Intersection,Food,Boat or Ferry,Wings Joint,Food & Drink Shop,Flower Shop,Filipino Restaurant
3,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Bagel Shop,Cosmetics Shop,Hotel,Food Truck,Pizza Place,Middle Eastern Restaurant,Supermarket
4,Bay Terrace,Italian Restaurant,Supermarket,Donut Shop,Plaza,Insurance Office,Salon / Barbershop,Sushi Restaurant,Liquor Store,Shipping Store,Wings Joint
5,Bloomfield,Recreation Center,Discount Store,Theme Park,Bus Stop,Donut Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
6,Bulls Head,Bus Stop,Pizza Place,Chinese Restaurant,Japanese Restaurant,Pharmacy,Deli / Bodega,Grocery Store,Baseball Field,Mattress Store,Fast Food Restaurant
7,Butler Manor,Pool,Baseball Field,Convenience Store,Bus Stop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
8,Castleton Corners,Pizza Place,Ice Cream Shop,Grocery Store,Diner,Sandwich Place,Park,Burger Joint,Bank,Tattoo Parlor,Bagel Shop
9,Charleston,Big Box Store,Cosmetics Shop,Shoe Store,Women's Store,Burger Joint,Sporting Goods Shop,Pet Store,Mobile Phone Shop,Shopping Mall,Scenic Lookout


#### Now we can cluster the neighborhoods


In [81]:
# set number of clusters
kclusters = 5

staten_island_grouped_clustering = staten_island_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(staten_island_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 4, 4, 1, 4, 4, 4, 4])

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

staten_island_merged = Staten_Island_data

# merge staten_island_grouped with staten_island_data to add latitude/longitude for each neighborhood
staten_island_merged = staten_island_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').reset_index(drop=True)

staten_island_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,4,Bar,Deli / Bodega,Thai Restaurant,Donut Shop,Bus Stop,Steakhouse,Monument / Landmark,Scenic Lookout,Italian Restaurant,Tapas Restaurant
1,Staten Island,New Brighton,40.640615,-74.087017,1,Bus Stop,Park,Convenience Store,Construction & Landscaping,Bowling Alley,Playground,Discount Store,Chinese Restaurant,Deli / Bodega,Flower Shop
2,Staten Island,Stapleton,40.626928,-74.077902,4,Pizza Place,Cosmetics Shop,Sandwich Place,Restaurant,Discount Store,Bank,Spanish Restaurant,Fast Food Restaurant,Mexican Restaurant,Breakfast Spot
3,Staten Island,Rosebank,40.615305,-74.069805,4,Italian Restaurant,Grocery Store,Pizza Place,Filipino Restaurant,Breakfast Spot,Bus Stop,Mexican Restaurant,Cajun / Creole Restaurant,Eastern European Restaurant,Beach
4,Staten Island,West Brighton,40.631879,-74.107182,4,Coffee Shop,Ice Cream Shop,Pharmacy,Music Store,Bar,Italian Restaurant,Event Space,Pizza Place,Mexican Restaurant,Liquor Store


In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(staten_island_merged['Latitude'], staten_island_merged['Longitude'], staten_island_merged['Neighborhood'], staten_island_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
staten_island_merged.loc[staten_island_merged['Cluster Labels'] == 0, staten_island_merged.columns[[1] + list(range(5, staten_island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Todt Hill,Park,Wings Joint,Dry Cleaner,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space,Eastern European Restaurant


#### Todt Hill, cluster 0, has a Filipino restaurant as 6th most common venue, falafel restaurant as 8th most common and Eastern Europen as the 10th most common venue.

In [85]:
staten_island_merged.loc[staten_island_merged['Cluster Labels'] == 1, staten_island_merged.columns[[1] + list(range(5, staten_island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,New Brighton,Bus Stop,Park,Convenience Store,Construction & Landscaping,Bowling Alley,Playground,Discount Store,Chinese Restaurant,Deli / Bodega,Flower Shop
5,Grymes Hill,Bus Stop,Basketball Court,Dog Run,Wings Joint,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
8,Port Richmond,Pizza Place,Bus Stop,Rental Car Location,Mexican Restaurant,Donut Shop,Dessert Shop,Eastern European Restaurant,Flower Shop,Filipino Restaurant,Fast Food Restaurant
9,Mariner's Harbor,Bus Stop,Deli / Bodega,Italian Restaurant,Ice Cream Shop,Construction & Landscaping,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
24,Park Hill,Bus Stop,Gym / Fitness Center,Hotel,Coffee Shop,Athletics & Sports,Dry Cleaner,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
27,Arlington,Bus Stop,Grocery Store,American Restaurant,Intersection,Food,Boat or Ferry,Wings Joint,Food & Drink Shop,Flower Shop,Filipino Restaurant
29,Grasmere,Bus Stop,Food,Bank,Bagel Shop,Japanese Restaurant,Food & Drink Shop,Nail Salon,Restaurant,Juice Bar,Ice Cream Shop
44,Chelsea,Theater,Bus Stop,Steakhouse,Sandwich Place,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space,Eastern European Restaurant
45,Bloomfield,Recreation Center,Discount Store,Theme Park,Bus Stop,Donut Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
52,Randall Manor,Bus Stop,Deli / Bodega,Wings Joint,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space


In [86]:
staten_island_merged.loc[staten_island_merged['Cluster Labels'] == 2, staten_island_merged.columns[[1] + list(range(5, staten_island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Port Ivory,Bar,Wings Joint,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
15,Oakwood,Bar,Wings Joint,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space


In [87]:
staten_island_merged.loc[staten_island_merged['Cluster Labels'] == 3, staten_island_merged.columns[[1] + list(range(5, staten_island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Emerson Hill,Gym,Wings Joint,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space,Eastern European Restaurant,Dry Cleaner


In [88]:
staten_island_merged.loc[staten_island_merged['Cluster Labels'] == 4, staten_island_merged.columns[[1] + list(range(5, staten_island_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. George,Bar,Deli / Bodega,Thai Restaurant,Donut Shop,Bus Stop,Steakhouse,Monument / Landmark,Scenic Lookout,Italian Restaurant,Tapas Restaurant
2,Stapleton,Pizza Place,Cosmetics Shop,Sandwich Place,Restaurant,Discount Store,Bank,Spanish Restaurant,Fast Food Restaurant,Mexican Restaurant,Breakfast Spot
3,Rosebank,Italian Restaurant,Grocery Store,Pizza Place,Filipino Restaurant,Breakfast Spot,Bus Stop,Mexican Restaurant,Cajun / Creole Restaurant,Eastern European Restaurant,Beach
4,West Brighton,Coffee Shop,Ice Cream Shop,Pharmacy,Music Store,Bar,Italian Restaurant,Event Space,Pizza Place,Mexican Restaurant,Liquor Store
7,South Beach,Deli / Bodega,Pier,Beach,Athletics & Sports,Wings Joint,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
11,Castleton Corners,Pizza Place,Ice Cream Shop,Grocery Store,Diner,Sandwich Place,Park,Burger Joint,Bank,Tattoo Parlor,Bagel Shop
12,New Springville,Mobile Phone Shop,Pizza Place,Chinese Restaurant,Bagel Shop,Deli / Bodega,Ice Cream Shop,Soup Place,Sandwich Place,Restaurant,Coffee Shop
13,Travis,Deli / Bodega,Hotel,Bowling Alley,Gym / Fitness Center,Spanish Restaurant,Baseball Field,Park,Gym,Sports Club,Comedy Club
14,New Dorp,Italian Restaurant,Pizza Place,Bank,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Salon / Barbershop,Sushi Restaurant,Taco Place,Bakery
16,Great Kills,Bar,Italian Restaurant,Pizza Place,Cosmetics Shop,Grocery Store,Food & Drink Shop,Falafel Restaurant,Sandwich Place,Chinese Restaurant,Bagel Shop
